In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

In [50]:
# Generate dummy dataset
def generate_dummy_chess_data(num_samples):
    X = np.zeros((num_samples, 8, 8, 12), dtype=np.float32)
    y = np.zeros((num_samples, 1), dtype=np.int64)
    
    for i in range(num_samples):
        for row in range(8):
            for col in range(8):
                piece_type = random.randint(0, 11)
                X[i, row, col, piece_type] = 1
        y[i] = random.randint(0, 63)
    
    return X, y

# Prepare the dataset
num_samples = 1000  # Number of samples to generate
X_train, y_train = generate_dummy_chess_data(num_samples)

# Print the shapes of the generated dataset
print(f"Generated dataset shapes: X_train = {X_train.shape}, y_train = {y_train.shape}")

Generated dataset shapes: X_train = (1000, 8, 8, 12), y_train = (1000, 1)


In [51]:
X_train = np.random.rand(1000, 8, 8, 12)  # 1000 board states, 8x8 board, 12 piece types
y_train = np.random.randint(0, 64, (1000, 1))  # 1000 moves (0-63 for board positions)

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).squeeze()

# Permute the tensor to match PyTorch's expected input format
X_train_tensor = X_train_tensor.permute(0, 3, 1, 2)  # Change from (batch, height, width, channels) to (batch, channels, height, width)

In [52]:
class ChessMovePredictor(nn.Module):
    def __init__(self):
        super(ChessMovePredictor, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=12, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 8 * 8, 128)  # Flattened feature size
        self.fc2 = nn.Linear(128, 64)  # Output layer for 64 possible board positions

    def forward(self, x):
        # Forward pass through the network
        x = torch.relu(self.conv1(x))  # Apply ReLU activation after the first conv layer
        x = torch.relu(self.conv2(x))  # Apply ReLU activation after the second conv layer
        x = x.reshape(x.size(0), -1)

        x = torch.relu(self.fc1(x))  # Apply ReLU activation after the first fully connected layer
        x = self.fc2(x)  # Output layer (logits for classification)
        
        return x

In [53]:
# Initialize model, loss function, and optimizer
model = ChessMovePredictor()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [54]:
model.parameters

<bound method Module.parameters of ChessMovePredictor(
  (conv1): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
)>

In [55]:
# Training loop
num_epochs = 20
batch_size = 32

for epoch in range(num_epochs):
    permutation = torch.randperm(X_train_tensor.size()[0])
    
    for i in range(0, X_train_tensor.size()[0], batch_size):
        indices = permutation[i:i + batch_size]
        batch_x, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/20], Loss: 4.1590
Epoch [2/20], Loss: 4.1843
Epoch [3/20], Loss: 4.1504
Epoch [4/20], Loss: 4.1538
Epoch [5/20], Loss: 4.0642
Epoch [6/20], Loss: 4.0285
Epoch [7/20], Loss: 3.9135
Epoch [8/20], Loss: 3.8055
Epoch [9/20], Loss: 4.1392
Epoch [10/20], Loss: 3.3363
Epoch [11/20], Loss: 3.0902
Epoch [12/20], Loss: 2.6026
Epoch [13/20], Loss: 2.1160
Epoch [14/20], Loss: 1.8668
Epoch [15/20], Loss: 1.2836
Epoch [16/20], Loss: 0.7911
Epoch [17/20], Loss: 0.7942
Epoch [18/20], Loss: 0.5901
Epoch [19/20], Loss: 0.5085
Epoch [20/20], Loss: 0.2082


In [56]:
# Evaluate the model (simple accuracy for demonstration purposes)
with torch.no_grad():  # Disable gradient calculation
    outputs = model(X_train_tensor)  # Use X_train_tensor which is a torch.Tensor
    # print(torch.max(outputs, 1))
    # print(outputs)
    _, predicted = torch.max(outputs, 1)  # Get the index of the highest logit
    accuracy = (predicted == y_train_tensor).float().mean()  # Calculate accuracy
    print(f'Accuracy: {accuracy.item():.4f}')  # Print accuracy



Accuracy: 0.9960


In [59]:
import torch.onnx

# Assume `model` is your trained PyTorch model and `dummy_input` is a sample input tensor
dummy_input = torch.randn(1, 12, 8, 8)  # Input tensor shape for a chess board state
torch.onnx.export(model, dummy_input, "chess_move_predictor.onnx", input_names=['input'], output_names=['output'])